<img src="assets/logos/Logo_main.png" width=500></img>
# Inicio

El propósito de este día de juegos es aprender cómo desarrollar e implementar rápidamente modelos de aprendizaje de máquina en un entorno competitivo. Llevaremos a cabo esto desarrollando modelos para jugar el juego de la batalla naval.

## Descripción del juego y reglas
¡Tu equipo creará modelos de aprendizaje de máquina para jugar "AI Command" que es similar a la batalla naval! (como en "¡hundiste mi barco de guerra!") Cada equipo tendrá un tablero en el que colocarán sus barcos e intentarán adivinar la ubicación de los barcos del equipo contrario. Durante cada ronda, los equipos elegirán una ubicación en el tablero del equipo contrario, lo que se revelará como un acierto o un yerro. ¡El primer equipo en encontrar todos los barcos del equipo contrario gana!

Para jugar el juego, cada equipo deberá crear dos modelos de aprendizaje de máquina: un modelo de distribución que ubique los barcos en el tablero y un modelo de disparo que adivine las ubicaciones de los barcos del otro equipo. 

### Reglas
- __Tablero__: Cada equipo en cada juego colocará sus barcos en un tablero de juego rectangular y luego adivinará o disparará al tablero de barcos contrario. El tamaño del tablero te será dado. 
- __Distribución__:
    - Barcos: Los barcos pueden ser de __cualquier forma__ siempre que sigan las restricciones dadas para la situación. Los detalles de estas restricciones estarán en las Instrucciones de punto final de distribución
- __disparo__: Los equipos realizarán un disparo por vez hasta que suceda una de las siguientes cosas:
    1. un equipo hunde todos los barcos del equipo contrario
    2. Se alcanza a la cantidad máxima de rondas de disparos. La cantidad máxima de rondas de disparo es igual al área del tablero de juego. Por lo tanto, si un Tablero de juego es de 10x10, habrá un máximo de 100 disparos por equipo por juego
- __Puntuación__: se llevará un registro de la puntuación de los equipos a lo largo de tres dimensiones
    1. Juegos ganados
    1. Cantidad total de aciertos contra los adversarios 
    1. Cantidad total de aciertos de los adversarios

### Arquitectura

<img src="assets/build/architecture.png"></img>

El motor del juego funciona de la siguiente manera:
1. una alarma de cloudwatch activa el __launcher lambda__ cada 30 segundos (u otro intervalo de tiempo) 
2. el launcher lambda lee la __tabla dynamodb del equipo__ (la cual se te lee cuando inscribes a tu equipo) y escribe en el SQS del juego una configuración del juego para cada par de equipos.  
3. __Game lambda__ lee los objetos de la cola del juego y ejecuta cada juego   
    1. llamando al __punto final de distribución__ de cada equipo
    2. llamando repetidamente al __punto final de disparo__ de cada equipo y actualizando la puntuación
    3. cuando el juego termina, escribe los datos del juego en la __tabla dynamodb del juego__
    4. si hay algún error en los equipos, escribe el error en el __error sns topic__ del equipo
4. La tabla __game DynamoDb__ transmite al lambda de transmisión, el cual: 
    1. escribe los datos del juego un __kinesis firehose__ que escribe los datos en un __bucket S3__ (para que los equipos consuman y descarguen) y una base de datos __elasticsearch database__ (para que los administradores del juego depuren los errores).
        1. uando se escriben nuevos datos en el bucket s3 se activa un __SNS topic__ para notificar a los equipos de los nuevos datos
    2. escribe las actualizaciones de puntuaciones en un __redis cluster__

## Preparación
El éxito de tu equipo depende de:

- Su capacidad de trabajar juntos
- Su capacidad de implementar modelos rápidamente
- Usar el tiempo eficazmente
- Sus conocimientos de aprendizaje de máquina


###  Organiza a tu equipo
Para ayudar a tu equipo a trabajar juntos e implementar modelos rápidamente, recomendamos organizar a tu equipo en tres grupos: Operaciones, defensa y ataque. Cada grupo tendrá sus propias habilidades y responsabilidades, y trabajarán con los otros grupos para ganar el juego. 

1. __Grupo de defensa__: Responsable de investigar y desarrollar el punto final de distribución. Los miembros que tengan buenas habilidades en ciencias de datos o habilidades de programación en python tendrán un buen desempeño en este equipo.
1. __Grupo de ataque__: Responsable de investigar y desarrollar el punto final de disparo. Los miembros con buenas habilidades en aprendizaje de máquina tendrán un buen desempeño en este equipo.
1. __Operaciones__: Los miembros que no se adecuen bien a los grupos de ataque o defensa pueden trabajar en este grupo:
    1. Administración/implementación de la infraestructura
    2. Implementación de modelos
    3. comunicación de los errores y comentarios a los grupos de Distribución y Disparos

¡También tendrás que darle un nombre a tu equipo! 


### Planificación
Para ayudar a tu equipo a usar su tiempo eficazmente, les sugerimos realizar la planificación para cada equipo y cada grupo. También les recomendamos trabajar en "lotes de tamaño pequeño". Prueba hacer muchos cambios pequeños e implementaciones rápidamente en lugar de hacer pocas implementaciones más grandes.

<img src="assets/build/schedue.png" width="800"></img>


## Código 

### Organización
El código que necesitarás ya fue instalado en tu instancia del cuaderno de notas. El directorio raíz del proyecto está en /home/ec2-user/SageMaker/GameDayRepo y las rutas dadas en cualquier cuaderno de notas posterior serán relativas a este directorio. En la raíz del proyecto hay un par de directorios y archivos que se deben tener en cuenta


```
bin/ #A directory of scripts to help you with common tasks
├── athena_info.sh
├── register_outputs.sh
├── startPipeline.js
├── testLayout.py
├── testShoot.py
├── upload_code.sh
├── download_data.sh
└── upload_data.sh

layout-config.json # Deploy/Train config for layout endpoint
shoot-config.json  # Deploy/Train config for shoot endpoint

containers/ #contains all the code for you sagemaker training jobs and endpoints
├── Layout # code for the layout endpoint
│   ├── 2vs3
│   │   ├── host.py
│   │   ├── test.py
│   │   └── train.py
└── Shoot # code for the shoot endpoint
    ├── CNN # code for Deep learning CNN model
    │   ├── cnn.py
    │   ├── host.py
    │   ├── mock
    │   │   ├── data # directory to put sample data for local testing
    │   │   └── out
    │   ├── test.py
    │   ├── train.py
    ├── line # code for line shooting model
    │   ├── host.py
    │   ├── test.py
    │   └── train.py
    
notebooks/ # Jupyter notebooks/Tutorials for your team to help you through the day
├── Athena.ipynb
├── Endpoint_Reference.ipynb
├── defense
│   ├── Analytics.ipynb
│   └── getting_started.ipynb
├── getting_started.ipynb
├── offensive
│   ├── CNN.ipynb
│   └── getting_started.ipynb
└── opperations
    └── getting_started.ipynb
    
data/ # Directory where the game data is downloaded and saved to.
```

### Implementación
Cada grupo trabajará en una instancia del cuaderno de notas de SageMaker. Los siguientes diagramas muestran cómo cada envío o lectura del cuaderno de notas cambia al codecommit repo principal (este repositorio ya fue creado para ti)
<img src="assets/build/overview.png" width="600"></img>


## ¡Divídanse en grupos!
sepárense en tres grupos y trabajen en los tutoriales correspondientes

- [Cuaderno de notas de operaciones](./opperations/getting_started.ipynb)
- [Cuaderno de notas ataque](./offensive/getting_started.ipynb)
- [Cuaderno de notas defensa](./defense/getting_started.ipynb)
